# - Autor: 
### Borja Delgado González
# - Objetivo: 
### Proyecto consistente en descargar la información de videojuegos de la página VGChartz y almacenar los datos en un archivo .csv
---
##### El proyecto no pretende ser eficiente. Se trata de un proyecto para demostrar cómo obtener datos de una página web.
###### A lo largo del código se han implementando algunos "prints" para que se pueda ver el elementos por separado.

1. Primero importamos las librerías necesarias

In [2]:
# Librerías
import requests
import urllib
from bs4 import BeautifulSoup, element

import pandas as pd
import numpy as np
from random import randint
import time
from time import sleep
from IPython.display import HTML, clear_output
import base64

2. Creamos las variables necesarias para el proceso

In [3]:
# Variables
url1 = 'https://www.vgchartz.com/games/games.php?page='
url2 = '&results=1000&order=Sales&ownership=Both&direction=DESC&showtotalsales=1' # despues de results= se puede añadir el número que queramos para modificar el número de resultados por página.
url2 += '&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1'
url2 += '&showreleasedate=1&showlastupdate=0&showvgchartzscore=0&showcriticscore=0&showuserscore=0&showshipped=0'

paginas = 63 # 63 es el número de páginas de la tabla si se muestran 1000 resultados por página.
videojuego = []
plataforma = []
genero = []
editor = []
desarrollador = []
ventas_na = []
ventas_eu = []
ventas_jp = []
ventas_otras = []
ventas_tot = []
fecha_salida = []
peticiones = 0 # en el código hay implementada una línea que muestra el número de datos (juegos) recogidos. Se inicializa a 0 para poder darle seguimiento al proceso.

dict_csv = {
    'videojuego': videojuego,
    'plataforma': plataforma,
    'genero': genero,
    'editor': editor,
    'desarrollador': desarrollador,
    'ventas_na': ventas_na,
    'ventas_eu': ventas_eu,
    'ventas_jp': ventas_jp,
    'ventas_otras': ventas_otras,
    'ventas_tot': ventas_tot,
    'fecha_salida': fecha_salida
    }


3. Implementamos el código para el scraping

In [4]:
#Creamos una session de requests
session = requests.Session()


#Establecemos un for loop que va a recorrer el DOM de la página web y localizará los elementos <a> que tengan un enlace.
for pagina in range(1, paginas + 1):
    surl = url1 + str(pagina) + url2
    
    response = session.get(surl)

    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Almacenamos los elementos en una lista para implementar otro for loop.
    etiquetas_videojuegos = list(filter(lambda x: 'href' in x.attrs and x.attrs['href'].startswith('https://www.vgchartz.com/game/'),soup.find_all("a")))
    #print(etiquetas_videojuegos)

    for etiqueta in etiquetas_videojuegos:

      #if int(etiqueta.parent.parent.find_all('td')[0].string) > 62000:
        #Le damos seguimiento al proceso mostrando el número de peticiones realizadas.
        peticiones += 1
        print('Peticiones: {}'.format(peticiones))
        clear_output(wait = True)
        
        #Lo primero que recogemos es el nombre del videojuego.
        videojuego.append(etiqueta.contents[0][:-4])
        #print(etiqueta.contents)
        
        #A partir de cada tag, subimos dos niveles en el DOM hasta encontrar la fila de cada elemento de la tabla y poder ir extrayendo los datos uno a uno.
        data = etiqueta.parent.parent.find_all("td")

        plataforma.append(data[3].find('img').attrs['alt'])
        editor.append(data[4].string)
        desarrollador.append(data[5].string)

        #Las ventas se recogen con el como floats o np.nan si no existe valor para evitar errores.
        ventas_na.append(
            float(data[7].string[:-1]) if
            not data[7].string.startswith("N/A") else np.nan)
        ventas_eu.append(
            float(data[8].string[:-1]) if
            not data[8].string.startswith("N/A") else np.nan)
        ventas_jp.append(
            float(data[9].string[:-1]) if
            not data[9].string.startswith("N/A") else np.nan)
        ventas_otras.append(
            float(data[10].string[:-1]) if
            not data[10].string.startswith("N/A") else np.nan)
        ventas_tot.append(
            float(data[6].string[:-1]) if
            not data[6].string.startswith("N/A") else np.nan)
        
        #Se para el proceso para evitar el error 429.
        sleep(randint(1,3))

        #Dividimos el elemento "Release Date" de la página para quedarnos sólo con el año, compuesto por dos dígitos y lo almacenamos en una variable.
        lanzamiento = data[11].string.split()[-1]
        
        if lanzamiento.startswith('N/A'):
            fecha_salida.append('N/A')
        else:
            if int(lanzamiento) >= 80:
                anyo_lanzamiento = np.int32("19" + lanzamiento)
            else:
                anyo_lanzamiento = np.int32("20" + lanzamiento)
            fecha_salida.append(anyo_lanzamiento)
        
        #El género del juego no aparece en la tabla. Tenemos que buscarlo dentro de la página del juego.
        #Identificamos el link para entrar a la página y dentro de la nueva página identificamos dónde se encuentra el género del juego.
        #Dentro del div "gameGenInfoBox" hay 3 etiquetas <h2> siendo una de ellas el género. Lo localizamos y lo almacenamos en una variable.
        #Una vez localizado el <h2> del género, recogemos el dato de la siguiente etiqueta
        url_genero = etiqueta.attrs['href']
        link_genero = urllib.request.urlopen(url_genero).read()
        soup_genero = BeautifulSoup(link_genero, "html.parser")
      
        h2s = soup_genero.find("div", {"id": "gameGenInfoBox"}).find_all('h2')
        
        etiqueta_genero = element.Tag
        for h2 in h2s:
            if h2.string == 'Genre':
                etiqueta_genero = h2
                #print(etiqueta_genero)
        genero.append(etiqueta_genero.next_sibling.string)
        #print(etiqueta_genero.next_sibling.string)



KeyboardInterrupt: 

4. Creamos un csv a partir de las listas de datos

In [ ]:
df = pd.DataFrame(dict_csv)

df.to_csv('ventas_videojuegos.csv')